# PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook
for the following questions.

In [1]:
import sqlite3
import pandas as pd
dbpath = '08-SQL & Database\sqlite_db_pythonsqlite.db'
connect = sqlite3.connect(dbpath)

## Question 10:
Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! 

In [2]:
query = """SELECT
	r.Facility as Facility,
	r.Revenue as Revenue
FROM	
	(SELECT f.name AS Facility,
			SUM(gb.slots * f.guestcost) AS Revenue
	FROM Bookings gb
	INNER JOIN Facilities f ON f.facid = gb.facid
	WHERE gb.memid = 0
	UNION ALL
	SELECT 	f.name AS Facility,
			SUM(mb.slots * f.membercost) AS Revenue
	FROM Bookings mb
	INNER JOIN Facilities f ON f.facid = mb.facid
	WHERE mb.memid != 0
	GROUP BY Facility) as r
WHERE r.Revenue < 1000
ORDER BY Revenue"""
cur = connect.cursor()
cur.execute(query)
result = cur.fetchall()
df = pd.DataFrame.from_records(result, columns=['Facility', 'Revenue'])
cur.close()
df


,Facility,Revenue
0,Badminton Court,0.0
1,Pool Table,0.0
2,Snooker Table,0.0
3,Table Tennis,0.0
4,Massage Room 2,534.6


## Question 11:
Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [3]:
query = """SELECT
	m1.surname AS Surname,
	m1.firstname AS Firstname,
	m2.surname AS RecommenderSurname,
	m2.firstname AS RecommenderFirstname
FROM Members AS m1
INNER JOIN Members as m2
ON m1.recommendedby = m2.memid
WHERE m1.memid != 0 AND m2.memid != 0
ORDER BY m1.Surname, m1.Firstname
"""
cur = connect.cursor()
cur.execute(query)
result = cur.fetchall()
df = pd.DataFrame.from_records(result, columns=['Surname', 'Firstname', 'Recommender Surname', 'Recommender Firstname'])
cur.close()
df

,Surname,Firstname,Recommender Surname,Recommender Firstname
0,Bader,Florence,Stibbons,Ponder
1,Baker,Anne,Stibbons,Ponder
2,Baker,Timothy,Farrell,Jemima
3,Boothe,Tim,Rownam,Tim
4,Butters,Gerald,Smith,Darren
5,Coplin,Joan,Baker,Timothy
6,Crumpet,Erica,Smith,Tracy
7,Dare,Nancy,Joplette,Janice
8,Genting,Matthew,Butters,Gerald
9,Hunt,John,Purview,Millicent


## Question 12:
Find the facilities with their usage by member, but not guests 

In [4]:
query = """
SELECT	
	f.name as Facility,
	m.surname || m.firstname as Member,
	SUM(b.slots) as Utilization
FROM Bookings b
INNER JOIN Facilities f ON f.facid = b.facid
INNER JOIN Members m ON m.memid = b.memid 
WHERE b.memid != 0
GROUP BY Facility, Member
	
"""
cur = connect.cursor()
cur.execute(query)
result = cur.fetchall()
df = pd.DataFrame.from_records(result, columns=['Facility', 'Member', 'Usage'])
cur.close()
df


,Facility,Member,Usage
0,Badminton Court,BaderFlorence,27
1,Badminton Court,BakerAnne,30
2,Badminton Court,BakerTimothy,21
3,Badminton Court,BootheTim,36
4,Badminton Court,ButtersGerald,63
...,...,...,...
197,Tennis Court 2,SmithDarren,57
198,Tennis Court 2,SmithJack,3
199,Tennis Court 2,SmithTracy,6
200,Tennis Court 2,StibbonsPonder,96


## Question 13
Find the facilities usage by month, but not guests 

In [7]:
query = """
SELECT	
	f.name as Facility,
	strftime('%m',b.starttime) AS Month,
	SUM(b.slots) AS Utilization
FROM Bookings b
INNER JOIN Facilities f ON f.facid = b.facid
INNER JOIN Members m ON m.memid = b.memid 
WHERE b.memid != 0
GROUP BY Facility, Month
	
"""
cur = connect.cursor()
cur.execute(query)
result = cur.fetchall()
df = pd.DataFrame.from_records(result, columns=['Facility', 'Month', 'Usage'])
cur.close()
df

,Facility,Month,Usage
0,Badminton Court,07,165
1,Badminton Court,08,414
2,Badminton Court,09,507
3,Massage Room 1,07,166
4,Massage Room 1,08,316
5,Massage Room 1,09,402
6,Massage Room 2,07,8
7,Massage Room 2,08,18
8,Massage Room 2,09,28
9,Pool Table,07,110
